<a href="https://colab.research.google.com/github/Poonam-Majhi/CodeClause_Movie_Recommendation_System/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
credits_df = pd.read_csv('tmdb_credits.csv')
movies_df = pd.read_csv('tmdb_movies.csv')

In [ ]:
movies_df.head()

In [ ]:
credits_df.drop(credits_df.columns[4:1261], axis=1, inplace=True)

In [ ]:
credits_df.head()

In [ ]:
credits_df.columns = ['movie_id','title','cast','crew']
movies_df = movies_df.merge(credits_df, on="movie_id")

In [ ]:
movies_df.head()

In [ ]:

C = movies_df["vote_average"].mean()
m = movies_df["vote_count"].quantile(0.9)

print("C: ", C)
print("m: ", m)

new_movies_df = movies_df.copy().loc[movies_df["vote_count"] >= m]
print(new_movies_df.shape)

In [ ]:
def weighted_rating(x, C=C, m=m):
    v = x["vote_count"]
    R = x["vote_average"]

    return (v/(v + m) * R) + (m/(v + m) * C)

In [ ]:
new_movies_df["score"] = new_movies_df.apply(weighted_rating, axis=1)
new_movies_df = new_movies_df.sort_values('score', ascending=False)

new_movies_df[["title_x", "vote_count", "vote_average", "score"]].head(10)

In [ ]:

def plot():
    popularity = movies_df.sort_values("popularity", ascending=False)
    plt.figure(figsize=(12, 6))
    plt.barh(popularity["title_x"].head(10), popularity["popularity"].head(10), align="center", color="skyblue")
    plt.gca().invert_yaxis()
    plt.title("Top 10 movies")
    plt.xlabel("Popularity")
    plt.show()
    

plot()

In [ ]:

print(movies_df["overview"].head(5))

In [ ]:
tfidf = TfidfVectorizer(stop_words="english")
movies_df["overview"] = movies_df["overview"].fillna("")

tfidf_matrix = tfidf.fit_transform(movies_df["overview"])
print(tfidf_matrix.shape)

In [ ]:

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cosine_sim.shape)

indices = pd.Series(movies_df.index, index=movies_df["title_x"]).drop_duplicates()
print(indices.head())

In [ ]:
def get_recommendations(title_x, cosine_sim=cosine_sim):

    idx = indices[title_x]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    

    movies_indices = [ind[0] for ind in sim_scores]
    movies = movies_df["title_x"].iloc[movies_indices]
    return movies

In [ ]:
print("################ Content Based Filtering - plot#############")
print()
print("Recommendations for Star Wars")
print(get_recommendations("Star Wars"))
print()
print("Recommendations for The Godfather")
print(get_recommendations("The Godfather"))

In [ ]:
features = ["cast", "crew", "keywords", "genres"]

for feature in features:
    movies_df[feature] = movies_df[feature]

movies_df[features].head(10)

In [ ]:
def get_director(x):
    for i in x:
        if i["job"] == "Director":
            return i["name"]
    return np.nan

In [ ]:
def get_list(x):
    if isinstance(x, list):
        names = [i["name"] for i in x]

        if len(names) > 3:
            names = names[:3]

        return names

    return []

In [ ]:
movies_df["director"] = movies_df["crew"]

features = ["cast", "keywords", "genres"]
for feature in features:
    movies_df[feature] = movies_df[feature]

In [ ]:
movies_df[['title_x', 'cast', 'director', 'keywords', 'genres']].head()

In [ ]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ""

In [ ]:
features = ['cast', 'keywords', 'director', 'genres']
for feature in features:
    movies_df[feature] = movies_df[feature].apply(clean_data)

In [ ]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])


movies_df["soup"] = movies_df.apply(create_soup, axis=1)
print(movies_df["soup"].head())

In [ ]:
count_vectorizer = CountVectorizer(stop_words="english")
count_matrix = count_vectorizer.fit_transform(movies_df["soup"])

print(count_matrix.shape)

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim2.shape)

movies_df = movies_df.reset_index()
indices = pd.Series(movies_df.index, index=movies_df['title_x'])

In [ ]:
print("################ Content Based System - metadata #############")
print("Recommendations for Star Wars")
print(get_recommendations("Star Wars", cosine_sim2))
print()
print("Recommendations for The Godfather")
print(get_recommendations("The Godfather", cosine_sim2))